In [1]:
## strip out extra lines between entries
## todo: convert to regex

ft = open("tables.txt", 'r', encoding="utf8")
ft_new = open("tables_clean.txt", 'w', encoding="utf8")

line1 = ft.readline()
line2 = ft.readline()
isempty1 = line1.isspace()
isempty2 = line2.isspace()

ft_new.write(line1)
for line3 in ft:
    isempty3 = line3.isspace()
    if (not isempty2) or isempty1 or isempty3:
        ft_new.write(line2)
    
    line1 = line2
    line2 = line3
    isempty1 = isempty2
    isempty2 = isempty3
    
ft_new.write(line3)

ft.close()
ft_new.close()

In [ ]:
## save each entry as separate text file
ft = open("tables_clean.txt", 'r')
l = []

    

In [37]:
## strip out quotations and save to new file
ft = open("tables_clean.txt", 'r', encoding='utf8')
ft_new = open("tables_noquotes.txt", 'w', encoding='utf8')
ft_quotes = open("quotations.txt", 'w')

line1 = ft.readline()

for line2 in ft:

    if "\u2014" in line2[0]: # \u2014 is em dash
#         print(line1, line2)
        ft_quotes.writelines([line1, line2, "\n"])
        line1 = ft.readline()
    else:  
        ft_new.write(line1)
        line1 = line2;

ft.close()
ft_new.close()
ft_quotes.close()

In [2]:
## strip out flavor affinities and save to new file



In [36]:
## input filename, output dictionary of {flavor: [text as list of lines]} pairs

ft = open("tables_clean.txt", 'r', encoding='utf8')

lines = ft.readlines()

flavor = lines[0].lower()
text = []
flavor_dict = {}

for i in range(1, len(lines)-1):
    cur = lines[i].rstrip()
    
    if cur.isupper() and lines[i-1].isspace() and lines[i+1].isspace():
        flavor_dict[flavor] = text
        flavor = cur.lower()
        text = []
    else:
        text.append(cur)
        
ft.close()

In [39]:
flavor_dict['apples']

['',
 '',
 'Season: autumn',
 'Taste: sweet, astringent',
 'Function: cooling',
 'Weight: medium',
 'Volume: quiet–moderate',
 'Techniques: bake, caramelize, deep-fry (e.g., as fritters), grill, poach, raw, sauté, stew',
 '',
 '',
 'allspice',
 'almonds',
 '',
 '',
 '',
 '',
 '',
 'The combination of apple and celery works. A tart green apple sorbet is not going to knock your socks off, because you are programmed to know it. When you add the flavor of celery, you get something new. I also love the flavors of apple and fennel together, especially in sorbet.',
 '—MICHAEL LAISKONIS, LE BERNARDIN (NEW YORK CITY)',
 '',
 '',
 '',
 'I make an apple confit of thinly sliced apples with cinnamon caramel powder layered between the apples and baked slowly. When the dish is served, next to the apples is a small pile of dates poached in syrup with vanilla. The other flavors on the plate are lemon confit, quince, raw apple with apple cider gelée, and ras el hanout (a Moroccan spice blend).',
 'For t